In [1]:
import mediapipe as mp
import cv2
import socket
import json

HOST = '127.0.0.1'  # Localhost
PORT = 65493  # Port to listen on

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((HOST, PORT))
server.listen(1)
print("Waiting for a connection...")
conn, addr = server.accept()
print(f"Connected by {addr}")

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils


cap = cv2.VideoCapture(1)
while cap.isOpened():
    ret , frame = cap.read()
    if not ret :
        print("can not recive frame")
    frame = cv2.resize(frame , (640,480))
    RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(RGB)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
                
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            y = 1.0 - index_finger_tip.y  # Flip the Y-axis

            finger_data = {'y': y}
            conn.sendall(json.dumps(finger_data).encode('utf-8'))
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cv2.imshow("Flappy Bird Game", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
     

cap.release()
cv2.destroyAllWindows()
conn.close()
server.close()


Waiting for a connection...
Connected by ('127.0.0.1', 53611)
